# Wedge Task 3- Building Summary Tables

This task takes our tables in GBQ and answers the following questions: 

•	How have our sales-by-day changed over the last few months?

•	What is our most popular item in each department?

•	Which owners spend the most per month in each department?

This is done by pulling the table infomration through querys into a relational database: Wedge.db in this case. 

In [1]:
#imports, imports, imports
from google.cloud import bigquery
from google.oauth2 import service_account
import random
import sqlite3
import os

In [2]:
# Linking to my permissions and GBQ project
service_path = "/Users/kayleeosentowski/Desktop/MSBA/ADA_21/Wedge/"
service_file = 'osentowski-project-3a5d8e60c1a3.json' # authentication information  
gbq_proj_id = 'osentowski-project' # project. 

# using my machine math and json to create my private key
private_key =service_path + service_file

In [3]:
# credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [4]:
# establish connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

## Sales by date by hour:

By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items . 

In [5]:
# this code is building a query called "salesdatehour" that is pulling from all the transArchive tables
# columns pulled are datetime (extracted hour), total as sales, pulling transaction numbers be looking for unique date, register, employee, and transaction
#and items by not counting void or returns

salesdatehour = '''SELECT (EXTRACT(date FROM datetime)) AS Date,
    (EXTRACT(HOUR FROM datetime)) AS Hour,
    SUM(total) as Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(Trans_Status = 'V' OR Trans_Status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `osentowski-project.Transaction.transArchive*`

    WHERE Card_No != 3
    AND Department != 0
    AND Department != 15
    AND Trans_Status != 'M'
    AND Trans_Status != 'C'
    AND Trans_Status != 'J'
    AND (Trans_Status = ''
    OR Trans_Status = ' '
    OR Trans_Status = 'V'
    OR Trans_Status = 'R')
    GROUP BY Date, Hour
    ORDER BY Date, Hour'''
datehour = client.query(salesdatehour,location="US")

In [6]:
headers = ["Date","Hour","Sales","Transactions","Items"]

In [7]:
# writing out to a txt file with the headers above and stringing together the query results
with open("salesdatehour.txt", 'w') as outfile:
    outfile.write(",".join(headers) + "\n")
    for row in datehour:
         outfile.write(",".join([str(item) for item in row])+ '\n')

In [8]:
#connecting to SQLite database "Wedge.db"
db = sqlite3.connect("Wedge.db")
cur = db.cursor()

#input results
input_file = "salesdatehour.txt"

#building schema
cur.execute('''DROP TABLE IF EXISTS Daily_Hourly_Sales''') #remove table if it exists already
#creating table
cur.execute('''CREATE TABLE Daily_Hourly_Sales (
    Date TIMESTAMP,
    Hour TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items INTEGER)''')

# input the data into the table in the database
with open(input_file,'r', encoding="Latin-1") as ifile :
    for idx, line in enumerate(ifile) :
        line = line.strip().split(",")
        cur.execute('''
            INSERT INTO Daily_Hourly_Sales(Date,Hour,Sales,Transactions,Items)
            VALUES (?,?,?,?,?)''', line)

db.commit()

## Sales by owner by year by month:
A file that has the following columns: card_no, year, month, sales, transactions, and items. 

In [9]:
#building query from all of the GBQ transArchive files
#This is looking at card_no to count as owner
#I'm including card # 3 to see how many sales go to non-owners

query_owneryearmonth = '''SELECT card_no AS Owner,
    (EXTRACT(Year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    SUM(total) as Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(Trans_Status = 'V' OR Trans_Status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `osentowski-project.Transaction.transArchive*`

    WHERE Department != 0
    AND Department != 15
    AND Trans_Status != 'M'
    AND Trans_Status != 'C'
    AND Trans_Status != 'J'
    AND (Trans_Status = ''
    OR Trans_Status = ' '
    OR Trans_Status = 'V'
    OR Trans_Status = 'R')
    GROUP BY Card_No, Year, Month
    ORDER BY Card_No, Year, Month'''

owneryearmonth = client.query(query_owneryearmonth,location="US")

In [10]:
headers = ["Owner","Year","Month","Sales","Transactions","Items"]

In [11]:
#writing out to the file on my machine by stringing together the items in rows from the query results
with open("owneryearmonth.txt", 'w') as outfile:
    outfile.write(",".join(headers) + "\n")
    for row in owneryearmonth:
         outfile.write(",".join([str(item) for item in row])+ '\n')

In [14]:
#connecting to SQLite db
db = sqlite3.connect('Wedge.db')
cur = db.cursor()

# input the results of the first query
input_file2 = "owneryearmonth.txt"

cur.execute('''DROP TABLE IF EXISTS Monthly_Owner_Sales''') # remove the table if it already exists
# create the table in the db
cur.execute('''CREATE TABLE Monthly_Owner_Sales (
    Owner INTEGER,
    Year TIMESTAMP,
    Month TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items INTEGER)''')
# input the data into the table in the database
with open(input_file2, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Monthly_Owner_Sales (Owner, Year, Month, Sales, Transactions, Items)
        VALUES (?,?,?,?,?,?)''', line) #make sure that the ? match up with the column labels for value insertion
db.commit()

## Sales by product description by year by month:
A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.

In [15]:
# building query of all transArchive tables looking for the above columns. 
# Created a table in GBQ called DeptLookup using the apendix results for the Wedge Assignment
#Using a Left Outer Join to combine these descriptions on the Department number

query_proddescyearmonth = '''SELECT tr.Department, dl.dept_name,
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    UPC,
    Description,
    tr.Department AS Dept_Num,
    SUM(total) as Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(Trans_Status = 'V' OR Trans_Status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `osentowski-project.Transaction.transArchive*` AS tr
    
    LEFT OUTER JOIN `osentowski-project.Transaction.DeptLookup` AS dl ON tr.Department = dl.Department
    
    WHERE Card_No != 3
    AND tr.Department != 0
    AND tr.Department != 15
    AND Trans_Status != 'M'
    AND Trans_Status != 'C'
    AND Trans_Status != 'J'
    AND (Trans_Status = ''
    OR Trans_Status = ' '
    OR Trans_Status = 'V'
    OR Trans_Status = 'R')
    GROUP BY Year, Month, UPC, Description, Dept_Num, dl.dept_name
    ORDER BY Description, Year, Month DESC'''    
    
proddescyearmonth = client.query(query_proddescyearmonth,location="US")

In [16]:
headers = ["Department","Dept_Name","Year","Month","UPC","Description","Dept_no","Sales","Transactions","Items"]

In [17]:
with open("salesproddescyearmonth.txt", 'w') as outfile:
    outfile.write(",".join(headers) + "\n")
    for row in proddescyearmonth:
         outfile.write(",".join([str(item) for item in row])+ '\n')

In [19]:
#connecting to SQLite db
db = sqlite3.connect('Wedge.db')
cur = db.cursor()

#input results from query
input_file3 = "salesproddescyearmonth.txt"

cur.execute('''DROP TABLE IF EXISTS Prod_Desc_Year_Month''') #makes sure it isn't already done

#creating table in db
cur.execute('''CREATE TABLE Prod_Desc_Year_Month (
    Department FLOAT,
    Dept_Name TEXT,
    Year TIMESTAMP,
    Month TIMESTAMP,
    UPC STRING,
    Description STRING,
    Dept_no INTEGER,
    Sales REAL,
    Transactions INTEGER,
    Items INTEGER)''')

#input data into db 
with open(input_file3, 'r', encoding = 'utf-8') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Prod_Desc_Year_Month (Department, Dept_Name, Year, Month, UPC, Description, Dept_no, Sales, Transactions, Items)
        VALUES (?,?,?,?,?,?,?,?,?,?)''', line)    
db.commit()

In [39]:
db.close()

## Nonsense.

I like to include sometimes the random chunks of code I worked on before coming to this conlusion. Obviously most of this didn't work but I like to look back on it to see what I could use for other stuff.  

In [7]:
# for idx,row in enumerate(result) :
#     print("On {0}, card_no = {1} spent {2:,.2f}.".format(row[0],row[1],row[2]))
#     if idx > 20 :
#         break

In [23]:
# import sqlite3
# from wedge_helper import *

# input_file = "SalesDateHour.csv"

# # Let's just open the file and read the first few lines to the screen.
# with open(input_file,'r',encoding="Latin-1") as ifile :
#     for idx, line in enumerate(ifile.readlines()) :
#         print(line.strip().split("\t"))
#         if idx > 3 :
#             break

['YearMonthDay,Hour,Spend,NumberofItems,NumberofTrans']
['2012-07-16,6,209.63,53,3']
['2012-09-17,6,3.5527136788005009e-15,67,4']
['2016-06-05,6,-2.1400000000000148,45,3']
['2016-06-17,6,0.0,8,2']


In [24]:
# db = sqlite3.connect("WedgeSmall.db") #':memory:') # Make this a directory + file if you want to store the results.
# cur = db.cursor()

# init_db(cur) # take a look at the .py file to see what happened here.

OperationalError: database is locked

In [25]:
# with open(input_file,'r') as ifile :
#     populate_db(db,ifile,delimiter=",",limit=None)

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 50, and there are 5 supplied.

In [14]:
#  result = cur.execute('''SELECT card_no,
#                                 date(datetime) as date,
#                                 sum(total) AS spend
#                          FROM transactions
#                          WHERE trans_type = "I"
#                          GROUP BY card_no, date
#                          ''')

# for idx,row in enumerate(result) :
#     print("On {1}, card_no = {0} spent {2:,.2f}.".format(row[0],row[1],row[2]))
#     if idx > 20 :
#         break

On 2013-07-01, card_no = 3 spent 6,260.81.
On 2013-07-01, card_no = 10089 spent 57.11.
On 2013-07-01, card_no = 10095 spent 27.49.
On 2013-07-01, card_no = 10166 spent 4.82.
On 2013-07-01, card_no = 10286 spent 43.84.
On 2013-07-01, card_no = 10430 spent 77.12.
On 2013-07-01, card_no = 10478 spent 198.90.
On 2013-07-01, card_no = 10492 spent 19.76.
On 2013-07-01, card_no = 10499 spent 292.09.
On 2013-07-01, card_no = 10504 spent 4.29.
On 2013-07-01, card_no = 10510 spent 11.60.
On 2013-07-01, card_no = 10513 spent 4.45.
On 2013-07-01, card_no = 10527 spent 8.49.
On 2013-07-01, card_no = 10605 spent 81.45.
On 2013-07-01, card_no = 10613 spent 30.76.
On 2013-07-01, card_no = 10629 spent 20.71.
On 2013-07-01, card_no = 10656 spent 6.48.
On 2013-07-01, card_no = 10674 spent 49.28.
On 2013-07-01, card_no = 10682 spent 58.45.
On 2013-07-01, card_no = 10839 spent 13.77.
On 2013-07-01, card_no = 10959 spent 75.77.
On 2013-07-01, card_no = 11003 spent 12.27.


In [15]:
# result = cur.execute('''SELECT card_no,
#                                    department,
#                                    substr(date(datetime),1,4) AS year,
#                                    substr(date(datetime),6,2) AS month,
#                                    sum(total) AS spend,
#                                    count(distinct(date(datetime) || register_no ||
#                                            emp_no || trans_no)) as Transactions,
#                                    sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items
#                                    FROM transactions
#                                    WHERE department != 0 and
#                                         department != 15 and
#                                         trans_status != 'M' and
#                                         trans_status != 'C' and
#                                         trans_status != 'J' and
#                                        (trans_status = '' or 
#                                         trans_status = ' ' or 
#                                         trans_status = 'V' or 
#                                         trans_status = 'R') and card_no = 18736
#                           GROUP BY card_no, department, year, month
#                           ORDER BY year, month''')

In [17]:
# db.commit()

In [19]:
# for row in result :
#     print(row)

In [20]:
# for idx,row in enumerate(result) :
#     print(row)
#     if idx > 10 :
#         break

#1.	Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items . 

#2.	Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items. 

#3.	Sales by product description by year by month: A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.

In [29]:
import sqlite3
from wedge_helper import *

input_file = "SalesDateHour.csv"

#%%timeit # uncomment to test the execution time of the cell
db = sqlite3.connect("Wedge2.db") # this is the only change!
cur = db.cursor()
    
cur.execute('''DROP TABLE IF EXISTS transactions''')
cur.execute('''CREATE TABLE transactions (
    datetime TIMESTAMP,
    register_no INTEGER,
    emp_no INTEGER,
    trans_no INTEGER,
    upc INTEGER,
    description TEXT,
    trans_type TEXT,
    trans_subtype TEXT,
    trans_status TEXT,
    department INTEGER,
    quantity REAL,
    Scale INTEGER,
    cost REAL,
    unitPrice REAL,
    total REAL,
    regPrice REAL,
    altPrice REAL,
    tax INTEGER,
    taxexempt INTEGER,
    foodstamp INTEGER,
    wicable INTEGER,
    discount REAL,
    memDiscount REAL,
    discountable INTEGER,
    discounttype INTEGER,
    voided INTEGER,
    percentDiscount REAL,
    ItemQtty REAL,
    volDiscType INTEGER,
    volume INTEGER,
    VolSpecial REAL,
    mixMatch INTEGER,
    matched INTEGER,
    memType INTEGER,
    staff INTEGER,
    numflag INTEGER,
    itemstatus INTEGER,
    tenderstatus INTEGER,
    charflag TEXT,
    varflag INTEGER,
    batchHeaderID INTEGER,
    local INTEGER,
    organic INTEGER,
    display INTEGER,
    receipt INTEGER,
    card_no INTEGER,
    store INTEGER,
    branch INTEGER,
    match_id INTEGER,
    trans_id INTEGER)''')

with open(input_file,'r',encoding="Latin-1") as ifile :
    next(ifile)
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split("\t")
        cur.execute('''
            INSERT INTO transactions (datetime,register_no,emp_no,
              trans_no,upc,description,trans_type,trans_subtype,
              trans_status,department,quantity,Scale,cost,unitPrice,
              total,regPrice,altPrice,tax,taxexempt,foodstamp,wicable,
              discount,memDiscount,discountable,discounttype,voided,percentDiscount,
              ItemQtty,volDiscType,volume,
              VolSpecial,mixMatch,matched,memType,staff,numflag,itemstatus,tenderstatus,
              charflag,varflag,batchHeaderID,local,organic,display,receipt,
              card_no,store,branch,match_id,trans_id)
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
            ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
            ?,?,?,?,?,?,?,?,?,?,?,?,?)''', line)
    
db.commit() # Best practice, but not required here

result = cur.execute('''SELECT card_no,
                                   department,
                                   substr(date(datetime),1,4) AS Year,
                                   substr(date(datetime),6,2) AS Month,
                                   sum(total) AS spend,
                                   count(distinct(date(datetime) || register_no ||
                                           emp_no || trans_no)) as Transactions,
                                   sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items
                                   FROM transactions
                                   WHERE department != 0 and
                                        department != 15 and
                                        trans_status != 'M' and
                                        trans_status != 'C' and
                                        trans_status != 'J' and
                                       (trans_status = '' or 
                                        trans_status = ' ' or 
                                        trans_status = 'V' or 
                                        trans_status = 'R')
                          GROUP BY card_no, department, year, month''')

for idx,row in enumerate(result) :
    print(row)
    if idx > 10 :
        break
        
db.close()

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 50, and there are 1 supplied.

In [ ]:
#1.	Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items . 
#Results saved as SalesDateHour.csv
SELECT DATE(datetime) AS YearMonthDay,
EXTRACT(HOUR FROM datetime) as Hour,
SUM(total) AS Spend,
count(trans_type = 'I') as NumberofItems,
count(distinct(DATE(datetime)||register_no||emp_no||trans_no)) as NumberofTrans
FROM `umt-msba.transactions.*`
Group by YearMonthDay, Hour

In [ ]:
#2.	Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items. 
#Results saved as OwnerYearMonth.csv
SELECT card_no AS Owner,
EXTRACT(YEAR FROM datetime) AS Year,
EXTRACT(MONTH FROM datetime) AS Month,
SUM(total) AS Sales,
count(trans_type = 'I') as Items,
count(distinct(DATE(datetime)||register_no||emp_no||trans_no)) as Transactions
FROM `umt-msba.transactions.*`
Group by card_no, Year, Month
Order by card_no, Year, Month

In [ ]:
#3.	Sales by product description by year by month: A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.
#Results saved as SalesDateHour.csv
SELECT upc AS ProductDesc,
EXTRACT(YEAR FROM datetime) AS Year,
EXTRACT(MONTH FROM datetime) AS Month,
description AS Descrip,
department AS DeptNum,
count(trans_type = 'I') as Items,
count(distinct(DATE(datetime)||register_no||emp_no||trans_no)) as Transactions
FROM `umt-msba.transactions.*`
Group by upc, Year, Month, description, department
Order by upc, Year, Month
#Need to include department name, this is in a table in the appendix. Need to do a join but how?